In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, LSTM, Activation, TimeDistributed, RepeatVector, Input

Using TensorFlow backend.
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lab606a/.virtualenvs

In [2]:
import tensorflow as tf
# set GPU memory
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
dataset = pd.read_csv('/home/lab606a/ML/trajectories/fixed/classification/Lebeled_by_Kmeans/dataset20200320.csv', header=None)
dataset = dataset.fillna(0)
dataset = np.array(dataset)
dataset.shape

(4000, 93)

In [4]:
dataset = dataset.reshape(dataset.shape[0], int(dataset.shape[1]/3), 3)
dataset.shape

(4000, 31, 3)

In [18]:
#input_layer = Input(shape=(93,)) ##for Dense
input_layer = Input(shape=(31,3))

In [29]:
encoded = LSTM(64, activation='linear', return_sequences=True)(input_layer)
#encoder = LSTM(128, activation='linear', return_sequences=True)(encoder)
encoded = LSTM(16, activation='linear')(encoded)


## Dense
#encoded = Dense(64, activation='linear')(input_layer)
#encoded = Dense(16, activation='linear')(encoded)

In [30]:
decoded = RepeatVector(dataset.shape[1])(encoded)
#decoded = LSTM(16, activation='linear', return_sequences=True)(decoded)
decoded = LSTM(64, activation='linear', return_sequences=True)(decoded)
#decoder = LSTM(3, activation='linear', return_sequences=True)(decoder)
decoded = TimeDistributed(Dense(3))(decoded)

## Dense
#decoded = Dense(64, activation='linear')(encoded)
#decoded = Dense(93, activation='linear')(decoded)

In [31]:
encoder = Model(input_layer, encoded)

In [32]:
AutoEncoder = Model(input_layer, decoded)
AutoEncoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 31, 3)             0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 31, 64)            17408     
_________________________________________________________________
lstm_17 (LSTM)               (None, 16)                5184      
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 31, 16)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 31, 64)            20736     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 31, 3)             195       
Total params: 43,523
Trainable params: 43,523
Non-trainable params: 0
_________________________________________________________________


In [33]:
AutoEncoder.layers[2].output

<tf.Tensor 'lstm_17/TensorArrayReadV3:0' shape=(?, 16) dtype=float32>

In [35]:
decoded_input = Input(shape=(16,))
decoded_layer = AutoEncoder.layers[3](decoded_input)
decoded_layer = AutoEncoder.layers[4](decoded_layer)
decoder = Model(decoded_input, decoded_layer)

## Dense
#decoded_input = Input(shape=(16,))
#decoded_layer = AutoEncoder.layers[3](decoded_input)
#decoded_layer = AutoEncoder.layers[4](decoded_layer)
#decoder = Model(decoded_input, decoded_layer)

In [36]:
#Encoder = Model(input_layer, encoder)
#Encoder.summary()

In [37]:
AutoEncoder.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
AutoEncoder.fit(dataset, dataset, batch_size=400, epochs=200, shuffle=True)

Epoch 1/200
4000/4000 [==============================] - 2s 524us/step - loss: 2242.3607 - acc: 0.3570
Epoch 2/200
4000/4000 [==============================] - 1s 173us/step - loss: 1853.3851 - acc: 0.3198
Epoch 3/200
4000/4000 [==============================] - 1s 174us/step - loss: 1779.3619 - acc: 0.2920
Epoch 4/200
4000/4000 [==============================] - 1s 175us/step - loss: 1448.3750 - acc: 0.2982
Epoch 5/200
4000/4000 [==============================] - 1s 174us/step - loss: 1587.8602 - acc: 0.2872
Epoch 6/200
4000/4000 [==============================] - 1s 177us/step - loss: 1530.1342 - acc: 0.2763
Epoch 7/200
4000/4000 [==============================] - 1s 174us/step - loss: 1156.1580 - acc: 0.2815
Epoch 8/200
4000/4000 [==============================] - 1s 175us/step - loss: 895.8396 - acc: 0.3087
Epoch 9/200
4000/4000 [==============================] - 1s 179us/step - loss: 899.4442 - acc: 0.3280
Epoch 10/200
4000/4000 [==============================] - 1s 176us/step - l

4000/4000 [==============================] - 4s 963us/step - loss: 54.3155 - acc: 0.4891
Epoch 82/200
4000/4000 [==============================] - 4s 973us/step - loss: 54.2698 - acc: 0.4522
Epoch 83/200
4000/4000 [==============================] - 4s 975us/step - loss: 54.7520 - acc: 0.4500
Epoch 84/200
4000/4000 [==============================] - 4s 985us/step - loss: 54.5626 - acc: 0.4604
Epoch 85/200
4000/4000 [==============================] - 4s 984us/step - loss: 54.5935 - acc: 0.4447
Epoch 86/200
4000/4000 [==============================] - 4s 971us/step - loss: 54.5258 - acc: 0.4564
Epoch 87/200
4000/4000 [==============================] - 4s 979us/step - loss: 54.3457 - acc: 0.4539
Epoch 88/200
4000/4000 [==============================] - 4s 958us/step - loss: 54.3989 - acc: 0.4552
Epoch 89/200
4000/4000 [==============================] - 4s 982us/step - loss: 54.3233 - acc: 0.4623
Epoch 90/200
4000/4000 [==============================] - 4s 970us/step - loss: 54.1494 - acc: 

4000/4000 [==============================] - 4s 965us/step - loss: 52.6949 - acc: 0.4607
Epoch 161/200
4000/4000 [==============================] - 4s 961us/step - loss: 52.9580 - acc: 0.4512
Epoch 162/200
4000/4000 [==============================] - 4s 962us/step - loss: 53.2301 - acc: 0.4579
Epoch 163/200
4000/4000 [==============================] - 4s 968us/step - loss: 52.9752 - acc: 0.4610
Epoch 164/200
4000/4000 [==============================] - 4s 967us/step - loss: 52.8698 - acc: 0.4755
Epoch 165/200
4000/4000 [==============================] - 4s 971us/step - loss: 52.8154 - acc: 0.4690
Epoch 166/200
4000/4000 [==============================] - 4s 963us/step - loss: 52.8220 - acc: 0.4761
Epoch 167/200
4000/4000 [==============================] - 4s 967us/step - loss: 52.7271 - acc: 0.4666
Epoch 168/200
4000/4000 [==============================] - 4s 981us/step - loss: 52.7145 - acc: 0.4706
Epoch 169/200
4000/4000 [==============================] - 4s 958us/step - loss: 52.708

In [44]:
x_test = dataset[2,:]
x_test = x_test.reshape(1,x_test.shape[0],x_test.shape[1])
x_test = x_test.reshape(1, x_test.shape[0])
x_test.shape

ValueError: cannot reshape array of size 93 into shape (1,1)

In [ ]:
x_test

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
a = AutoEncoder.predict(x_test)
a

In [ ]:
a = encoder.predict(x_test)
a

In [ ]:
b = decoder.predict(a)
b

In [ ]:
from sklearn.cluster import KMeans

In [39]:
n_clusters = 8

In [40]:
from keras.engine.topology import Layer, InputSpec
import keras.backend as K

In [41]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.
    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [42]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)

In [43]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)

NameError: name 'KMeans' is not defined

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')

In [ ]:
y_pred = kmeans.fit_predict(encoder.predict(dataset))

In [ ]:
y_pred

In [ ]:
y_pred.shape

In [ ]:
df = pd.DataFrame(y_pred)
df.to_csv('./label.csv',index=0 ,header=0)